<a href="https://colab.research.google.com/github/Inigofs/kschool-dl/blob/main/Tensorflow_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools > /dev/null 2>&1
!pip install tensorflow-gpu==2.0.0 > /dev/null 2>&1

In [ ]:
import tensorflow as tf
import numpy as np

Let's try to fit a parabollic function using 

$\displaystyle y = -2x^2 + x + 1$

In [ ]:
f = lambda x: 2*x**2 + x +1
x_train = np.linspace(-100,100,1000)
y_train = f(x_train)

x_test = np.linspace(-110,-100.01,10)
y_test = f(x_test)

# Model Definition

### Sequential API

In [ ]:
sequential_model = tf.keras.models.Sequential()
sequential_model.add(tf.keras.layers.Dense(64, input_shape=(1,), activation='relu'))
sequential_model.add(tf.keras.layers.Dense(32, activation='relu'))
sequential_model.add(tf.keras.layers.Dense(1))

In [ ]:
sequential_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                128       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,241
Trainable params: 2,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sequential_model.predict(x_test)

1/1 [==============================] - 0s 314ms/step


array([[11.72321  ],
       [11.604917 ],
       [11.486614 ],
       [11.36832  ],
       [11.250021 ],
       [11.1317215],
       [11.013423 ],
       [10.895128 ],
       [10.776829 ],
       [10.658533 ]], dtype=float32)

### Functional API

In [ ]:
x = tf.keras.layers.Input(shape=(1,))
dense_relu_64 = tf.keras.layers.Dense(64, activation='relu')(x)
dense_relu_32 = tf.keras.layers.Dense(32, activation='relu')(dense_relu_64)
y = tf.keras.layers.Dense(1)(dense_relu_32)

functional_model = tf.keras.Model(x, y)
functional_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 dense_3 (Dense)             (None, 64)                128       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,241
Trainable params: 2,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
functional_model.predict(x_test)

1/1 [==============================] - 0s 93ms/step


array([[35.328083],
       [34.971596],
       [34.615093],
       [34.258606],
       [33.90211 ],
       [33.54562 ],
       [33.18912 ],
       [32.832638],
       [32.476143],
       [32.119644]], dtype=float32)

### Model Subclassing

In [ ]:
class NN(tf.keras.Model):

    def __init__(self):
        super(NN, self).__init__()
        self.dense_relu_64 = tf.keras.layers.Dense(64, activation='relu')
        self.dense_relu_32 = tf.keras.layers.Dense(32, activation='relu')
        self.dense_linear_1 = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense_relu_64(inputs)
        x = self.dense_relu_32(x)
        x = self.dense_linear_1(x)
        return x

In [ ]:
subclassing = NN()
x_train_sub = np.expand_dims(x_test, axis=1)
x_test_sub = np.expand_dims(x_test, axis=1)
print(subclassing(x_test_sub))

tf.Tensor(
[[-1.6487293]
 [-1.6320944]
 [-1.615457 ]
 [-1.5988178]
 [-1.5821824]
 [-1.5655451]
 [-1.5489082]
 [-1.5322733]
 [-1.5156322]
 [-1.4989953]], shape=(10, 1), dtype=float32)


# Training Model Subclassing

### Fit

In [ ]:
subclassing.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.mean_squared_error)

In [ ]:
subclassing.fit(x_train_sub, y_train, batch_size=8, epochs=10, validation_split=.2)

Epoch 1/10
1/1 [==============================] - 1s 782ms/step - loss: 385150688.0000 - val_loss: 369690400.0000
Epoch 2/10
1/1 [==============================] - 0s 35ms/step - loss: 385065952.0000 - val_loss: 369611840.0000
Epoch 3/10
1/1 [==============================] - 0s 41ms/step - loss: 384981376.0000 - val_loss: 369533344.0000
Epoch 4/10
1/1 [==============================] - 0s 36ms/step - loss: 384896832.0000 - val_loss: 369455584.0000
Epoch 5/10
1/1 [==============================] - 0s 33ms/step - loss: 384813088.0000 - val_loss: 369382496.0000
Epoch 6/10
1/1 [==============================] - 0s 35ms/step - loss: 384734400.0000 - val_loss: 369309728.0000
Epoch 7/10
1/1 [==============================] - 0s 40ms/step - loss: 384656000.0000 - val_loss: 369236800.0000
Epoch 8/10
1/1 [==============================] - 0s 34ms/step - loss: 384577504.0000 - val_loss: 369163904.0000
Epoch 9/10
1/1 [==============================] - 0s 37ms/step - loss: 384499008.0000 - val_los

### tf.GradientTape

In [ ]:
def optimize(model, x, y):
    with tf.GradientTape() as tape:
        pred = model(x)
        loss = tf.reduce_mean(tf.keras.losses.MSE(pred, y))

    grads = tape.gradient(loss, model.trainable_weights)
    optimizer = tf.keras.optimizers.Adam()
    optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
    return model, loss

In [ ]:
subclassing = NN()
x_train_sub = np.expand_dims(x_train, axis=1)
epochs = 10
for i in range(epochs):
    subclassing, loss =  optimize(subclassing, x_train_sub, y_test)
    print(i, loss) 

0 tf.Tensor(484119175.3771312, shape=(), dtype=float64)
1 tf.Tensor(484059331.3859114, shape=(), dtype=float64)
2 tf.Tensor(483998228.5404097, shape=(), dtype=float64)
3 tf.Tensor(483936191.8448219, shape=(), dtype=float64)
4 tf.Tensor(483873979.53862655, shape=(), dtype=float64)
5 tf.Tensor(483814264.8612247, shape=(), dtype=float64)
6 tf.Tensor(483755282.5257721, shape=(), dtype=float64)
7 tf.Tensor(483696390.8391052, shape=(), dtype=float64)
8 tf.Tensor(483636901.8169471, shape=(), dtype=float64)
9 tf.Tensor(483576977.89095265, shape=(), dtype=float64)
